<a href="https://colab.research.google.com/github/raiakash203/Machine-Learning/blob/Dimension-Reduction/SoftmaxClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Basic Softmax classifier on mnist data using tensorflow.
Implementation followed from AppliedAI course

In [0]:
!pip install tensorflow==1.14

## SoftMaxClassfier

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("MNIST", one_hot=True)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
'''
import tensorflow as tf
import numpy as np

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
'''

In [0]:
'''
train_images = train_images.reshape(-1,784)
test_images = test_images.reshape(-1,784)
'''

In [0]:
'''
#For one hot encoding of the data
encoding_shape_train = (train_labels.size,train_labels.max()+1)
encoding_shape_test = (test_labels.size,test_labels.max()+1)

train_one_hot = np.zeros(encoding_shape_train)
test_one_hot = np.zeros(encoding_shape_test)

rows = np.arange(train_labels.size)
train_one_hot[rows,train_labels] = 1
rows = np.arange(test_labels.size)
test_one_hot[rows,test_labels] = 1

train_labels = train_one_hot
test_labels = test_one_hot

'''

In [0]:
print("The number of data points in train ",mnist.train.images.shape[0],"No of pixels: ",mnist.train.images.shape[-1])

In [0]:
print("The number of data points in test ",mnist.test.images.shape[0],"shape of labels: ",mnist.test.labels.shape[-1])

In [0]:
# The below gives the list of GPUs and CPUs devices

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [0]:
# X is a placeholder, as the name suggests it holds a space in the memory
X = tf.placeholder(tf.float32,[None,784])

# Varibales are same as that in any programing language
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [0]:
#The predicted values is denoted by y
y = tf.nn.softmax(tf.matmul(X,W)+b)

In [0]:
#y_ is the true label
y_ = tf.placeholder(tf.float32,[None,10])

#The error is given by cross_entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_* tf.log(y), reduction_indices=[1]))

In [0]:
#The train step to minimize the loss which is cross entropy in our case

train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

In [0]:
#Now launch a the model in a session
session = tf.InteractiveSession()

#Now we will initialize the variables we have created
tf.global_variables_initializer().run()

In [0]:
#With each step of the loop will have a minibatch of 100 random data pointsfrom training sets

for _ in range(1000):
  batch_xs,batch_ys = mnist.train.next_batch(100)
  session.run(train_step,feed_dict={X:batch_xs,y_:batch_ys})

In [0]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(session.run(accuracy,feed_dict = {X:mnist.test.images,y_:mnist.test.labels}))

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time


def plot_dynamics(x,y,y_1,ax,colors=['b']):
  ax.plot(x,y,'b',label='Train Loss')
  ax.plot(x,y_1,'r',label='Test Loss')
  if len(x)==1:
    plt.legend()
  fig.canvas.draw()

In [0]:
#Sumarizing everything into a single cell
train_epooch = 20
batch_size = 1200

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels=y_))
train_step = tf.train.GradientDescentOptimizer(0.02).minimize(cross_entropy)

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoochs')
ax.set_ylabel('Softmax cross entropy loss')

xs,ytrs,ytes = [],[],[]

for epooch in range(train_epooch):
  train_avg_er = 0 
  test_avg_er =  0
  total_batch = int(mnist.train.num_examples/batch_size)
  #Loop over all batches
  for i in range(total_batch):
    batch_xs,batch_ys = mnist.train.next_batch(batch_size)
    _,c = session.run([train_step,cross_entropy], feed_dict = {X:batch_xs , y_:batch_ys})
    train_avg_er += c/total_batch
    c = session.run(cross_entropy, feed_dict = {X:mnist.test.images, y_:mnist.test.labels})
    test_avg_er += c/total_batch
  xs.append(epooch)
  ytrs.append(train_avg_er)
  ytes.append(test_avg_er)
  plot_dynamics(xs,ytrs,ytes,ax)

plot_dynamics(xs,ytrs,ytes,ax)
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print("Accuracy: ", accuracy.eval({X:mnist.test.images,y_:mnist.test.labels}))